<h1>Branch and Rebase</h1>

In this notebook you will work with two separate repositories, one where you will use the `git merge` command and another where you will use `git rebase` command to merge your files. Once both commands complete successfully, you will compare the results across both repositories using `git log`

Start by creating a repository where you use the `merge` command

In [0]:
!git init merge_repo

In [0]:
%cd merge_repo

The repository will store a list of colors from the business leader.

In [0]:
%%bash
git config --global user.email "bill@initech.com"
git config --global user.name "Bill Lumberg"
echo "Red
White
Blue" > colors

In [0]:
%%bash
git add colors
git commit -m 'initial colors'

In production, you should expect `develop` branch that is separate from `master`. Here, to make the branch tree easier to explain, the issue specific branches are created directly from `master`

In [0]:
!git branch yellow
!git branch green

One of the developers works on resolving the issue by adding `yellow` to the file.

In [0]:
%%bash
git config --global user.email "peter@initech.com"
git config --global user.name "Peter Gibbons"

In [0]:
!git checkout yellow

In [0]:
%%bash
echo "Yellow" >> colors
cat colors

In [0]:
!git commit -am 'added yellow'

The other developer works on adding color `green`.

In [0]:
%%bash
git config --global user.email "michael@initech.com"
git config --global user.name "Michael Bolton"

In [0]:
!git checkout green

In [0]:
%%bash
echo "Green" >> colors
cat colors

In [0]:
!git commit -am 'added green'

In the order of the commits, yellow was added first, followed by green. Let's suppose that the results of the `green` branch need to be merged into `yellow` for the latter to be resolved.

In [0]:
!git checkout yellow

Just confirming the status of `HEAD`, you are on the `yellow` branch, trying to merge the `green` branch files into `yellow`

In [0]:
!git log --graph --decorate --all --oneline

The multiline log also confirms that `yellow` was committed before `green`.

In [0]:
!git log --graph --decorate --all

Since the are no untracked files and you just checked out `yellow`, the output of the `status` command confirms that the working tree is clean

In [0]:
!git status

The recursive (three way) merge is expected to result in a conflict since the same lines of the `colors` file were modified in both the `yellow` and the `green` branches.

In [0]:
!git merge green

In [0]:
!cat colors

At this point, the developer working on the merge must choose from one of the three options:
* pick the  lines from `HEAD` (i.e. `yellow`)
* pick the lines from `green`
* combine the lines from both `HEAD` and `green`

If the developer does not know who worked on a specific issue, it is possible to use `git blame` to find out who committed conflicting changes.

In [0]:
!git blame colors

At this point the developer should know that both `yellow` and `green` issues tried to add colors to the file, so the right option is to combine the results. 

In [0]:
%%writefile colors
Red
White
Blue
Yellow
Green

Once the file is modified, it should be re-staged for the commit using either an explicit `git add` or using `git commit -a` option.

In [0]:
!git add colors

In [0]:
!git commit -m 'merged colors'

Great! The output of `git log` confirms that the recursive merge completed as expected.

In [0]:
!git log --graph --decorate --all

The branching structure created using recursive merge maintains detailed records of the merge order. Detailed records are valuable for internal development purposes, such as trying to understand the history of the changes, including who made the merges and when. However some branches (e.g. `release`) in Git are used for code distribution to external teams. Having too much detail about recursive merges in a publicly facing branch can be distracting for someone just trying to understand what features were added and when. 

One of the advantages of `git rebase` is the ability to "clean up" the history of a branch.

In [0]:
%cd ..

To start, let's reproduce the state of the `colors` file used in the previous example in a separate repository called `rebase_repo`. The next cell also creates the two issue branches to add yellow and green colors.

In [0]:
%%bash
rm -rf rebase_repo
git init rebase_repo
cd rebase_repo
git config --global user.email "bill@initech.com"
git config --global user.name "Bill Lumberg"
echo "Red
White
Blue" > colors
git add colors
git commit -m 'initial colors'
git branch yellow
git branch green

In [0]:
%cd rebase_repo

Just as in the previous example, the `yellow` issue is resolved first.

In [0]:
%%bash
git config --global user.email "peter@initech.com"
git config --global user.name "Peter Gibbons"

In [0]:
!git checkout yellow

In [0]:
%%bash
echo "Yellow" >> colors
cat colors

In [0]:
!git commit -am 'added yellow'

Followed by the resolution of the `green` issue.

In [0]:
%%bash
git config --global user.email "michael@initech.com"
git config --global user.name "Michael Bolton"

In [0]:
!git checkout green

In [0]:
%%bash
echo "Green" >> colors
cat colors 

In [0]:
!git commit -am 'added green'

In [0]:
!git checkout yellow

In [0]:
!git log --graph --decorate --all --oneline

In [0]:
!git status

At this point, the state of this repo is identical to the state of the repo in the previous example.

Notice that the next cell is using `git rebase` instead of `git merge` that you have seen earlier. Remember that by default the rebase operation assumes that the rebase should done to the `HEAD`. In other words `git rebase <what> <to> ` uses `HEAD` as the default value of `<to>` unless another branch or a commit is specified explicitly as an argument to the `rebase` command.

In [0]:
!git rebase green

Not surprisingly, the rebase results in a merge conflict because the same lines were modified in the `colors` file across the `yellow` and `green` branches.

In [0]:
!cat colors

Just as in the last example, the merge conflict is resolved by using all the lines from all the modifications to the colors file.

In [0]:
%%writefile colors
Red
White
Blue
Yellow
Green

Before a rebase can complete, the modified files must be re-staged.

In [0]:
!git add colors

To complete the rebase use the `continue` argument as in the next cell.

In [0]:
!git rebase --continue

Note the linear and ordered structure of the resulting `git log` and compare that to the detailed branching history of the example where you used the `git merge` operation.

In [0]:
%cd ../rebase_repo
!git log --graph --decorate --all --oneline

In [0]:
%cd ../merge_repo
!git log --graph --decorate --all --oneline

You might expect that the order of the log is also the chronological order of the commits. The `rebase` log can be misleading. Take a look at the timestamps in the merge log, and notice that it correctly preserves the details of the changes. Although the `merge` log is more difficult to understand, it is more accurate.

In [0]:
%cd ../merge_repo
!git log --graph --decorate --all

In contrast, the "cleaned up" history in the `rebase` log reports that the resolution of the `yellow` branch happened after the resolution of the `green` branch. However checking the stamps shows that not to be the case.

In [0]:
%cd ../rebase_repo
!git log --graph --decorate --all

Copyright 2019 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License